In [23]:
import numpy as np
from torch import nn, optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch

In [24]:
# 训练集
train_dataset = datasets.MNIST(root='./',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)
# 测试集
test_dataset = datasets.MNIST(root='./',
                               train=False,
                               transform=transforms.ToTensor(),
                               download=True)     

In [25]:
# 批次大小
batch_size = 64

# 装载训练集
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True)
# 装载测试集
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=batch_size,
                          shuffle=True)

In [26]:
for i,data in enumerate(train_loader):
    inputs,labels = data
    # print(inputs)
    # print(labels)
    print(inputs.shape)
    print(labels.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [27]:
labels

tensor([0, 8, 9, 1, 1, 2, 7, 6, 1, 1, 0, 2, 0, 8, 9, 2, 4, 1, 6, 5, 0, 2, 2, 6,
        2, 9, 2, 6, 7, 4, 6, 2, 4, 5, 3, 0, 6, 2, 0, 0, 3, 7, 9, 9, 3, 2, 4, 8,
        9, 2, 3, 6, 9, 9, 0, 5, 7, 1, 8, 0, 2, 2, 8, 5])

In [28]:
len(train_loader)

938

In [29]:
# 定义网络结构
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # nn.Dropout(p=0)就代表不用Dropout
        # 需要注意的是, Dropout也不是用了效果的一定好
        self.conv1 = nn.Sequential(nn.Conv2d(1,32,5,1,2), nn.ReLU(), nn.MaxPool2d(2,2))
        self.conv2 = nn.Sequential(nn.Conv2d(32,64,5,1,2), nn.ReLU(), nn.MaxPool2d(2,2))
        self.fc1 = nn.Sequential(nn.Linear(64*7*7,1000), nn.Dropout(p=0.5), nn.ReLU())
        self.fc1 = nn.Sequential(nn.Linear(1000,10), nn.Softmax(dim=1))
    
    def forward(self, x):
        # [64, 1, 28, 28]
        x = self.conv1(x)
        x = self.conv2(x)
        # (64,64,7,7)
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [30]:
LR = 0.0003
# 定义模型
model = Net()
# 定义损失函数
mse_loss = nn.CrossEntropyLoss()
# 定义优化器
optimizer = optim.Adam(model.parameters(), LR)

In [31]:
def train():
    # 训练状态
    model.train()
    for i, data in enumerate(train_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # 获得模型预测结构 (64,10)
        out = model(inputs)
        # 交叉熵代价函数out(batch,C), labels(batch)
        loss = mse_loss(out, labels)
        # 梯度清0
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 修改权值
        optimizer.step()
        
def test():
    # 测试状态
    model.eval()
    correct = 0
    for i, data in enumerate(test_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # 获得模型预测结构（64,10）
        out = model(inputs)
        # 获得最大值，以及最大值的位置
        _, priedicted = torch.max(out, 1)
        # 预测正确的数量
        correct += (predicted == labels).sum()
    print("Test acc:{0}".format(correct.item()/len(test_dataset)))
    
    correct = 0
    for i, data in enumerate(test_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # 获得模型预测结构（64,10）
        out = model(inputs)
        # 获得最大值，以及最大值的位置
        _, priedicted = torch.max(out, 1)
        # 预测正确的数量
        correct += (predicted == labels).sum()
    print("Test acc:{0}".format(correct.item()/len(train_dataset)))

In [13]:
for epoch in range(20):
    print('epoch:', epoch)
    train()
    test()

epoch: 0


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x3136 and 1000x10)